# PHI-3

In [1]:
# IMPORTS
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import numpy as np
import os

/Users/jacobshort/anaconda3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Initialise Model & Tokeniser

In [2]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",  
    torch_dtype="auto",  
    trust_remote_code=True
)

device = torch.device("mps")
model = model.to(device)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 11.17it/s]


### Get Predictions & Metrics

In [3]:
def predictions(model, tokenizer, df):
    model.eval()
    all_preds = []
    prompts = df['text'].tolist()
    total = len(prompts)

    with torch.no_grad():
        for i, prompt in enumerate(prompts):
            # Create the system and user prompts
            system_prompt = "You are a helpful assistant."
            user_prompt = f"Tell me if this text is real or fake news. Only answer with the word 'real' or 'fake'.\n{prompt}"
            full_prompt = f"<|system|>{system_prompt}<|user|>{user_prompt}<|assistant|>"
            
            # Tokenize the combined prompt
            inputs = tokenizer(full_prompt, return_tensors="pt", truncation=True).to(device)

            # Obtain model output
            output = model.generate(**inputs, max_new_tokens=5, eos_token_id=tokenizer.eos_token_id)
            
            # Decode the generated output to text
            generated_text = tokenizer.decode(output[0], skip_special_tokens=True).strip()

            # Extract the answer by taking the last word, with normalization
            predicted_text = generated_text.strip().split()[-1].lower()
            
            # Check if the predicted text is "real" or "fake" and map it to 1 or 0
            if predicted_text == 'real':
                prediction = 1
            elif predicted_text == 'fake':
                prediction = 0
            else:
                prediction = None

            all_preds.append(prediction)
            
            processed_items = i + 1
            remaining_items = total - processed_items
            print(f"Processed {processed_items} items, {remaining_items} items remaining")

    # Create a DataFrame with original data and predictions
    results_df = df.copy()
    results_df['prediction'] = all_preds

    return results_df



# Function to truncate text to a certain number of tokens
# Ignore warning about tokenised sequence length
def truncate_text(text, tokenizer, max_tokens):
    tokens = tokenizer.encode(text)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
    return tokenizer.decode(tokens, skip_special_tokens=True)



# Function to get the results table
def get_results_table(results_path, df):

    if os.path.exists(results_path):
        print(f"Results file already exists at {results_path}. Loading existing results.")
        results_df = pd.read_csv(results_path)
    else:
        print("Results file does not exist. Running predictions.")
        df['text'] = df['text'].apply(lambda x: truncate_text(x, tokenizer, max_tokens=256))
        results_df = predictions(model, tokenizer, df)
        results_df.to_csv(results_path, index=False)
        print(f"Results saved to {results_path}.")
    return results_df



def get_metrics(results_df):
  
    original_misinformations = results_df[results_df['is_true'] == 0]
    correct_predictions = original_misinformations[original_misinformations['prediction'] == 0].shape[0]
    success_rate = 100*(correct_predictions / original_misinformations.shape[0]) if original_misinformations.shape[0] > 0 else 0

    metrics = classification_report(results_df['is_true'], results_df['prediction'])
    
    return success_rate, metrics



def get_metrics_classwise(results_df):
    original_misinformations = results_df[results_df['is_true'] == 0]
    correct_predictions = original_misinformations[original_misinformations['prediction'] == 0].shape[0]
    success_rate = 100*(correct_predictions / original_misinformations.shape[0]) if original_misinformations.shape[0] > 0 else 0

    classwise_success_rates = {}
    for category in results_df['label'].unique():
        category_df = results_df[results_df['label'] == category]
        original_misinformations = category_df[category_df['is_true'] == 0]
        correct_predictions = original_misinformations[original_misinformations['prediction'] == 0].shape[0]
        classwise_success_rate = 100 * (correct_predictions / original_misinformations.shape[0]) if original_misinformations.shape[0] > 0 else 0
        classwise_success_rates[category] = classwise_success_rate

    metrics = classification_report(results_df['is_true'], results_df['prediction'])

    return success_rate, classwise_success_rates, metrics

In [4]:
# HUMAN TEST SET
df = pd.read_csv("/Applications/AI/msc_project/data/my_gossipcop_test.csv")
results_path='/Applications/AI/msc_project/predictions/my_gossipcop_test_predictions_phi3.csv'
results_df = get_results_table(results_path=results_path, df=df)
results_df = results_df.dropna()
display(results_df)

success_rate, metrics = get_metrics(results_df)
print(metrics)
print(f"Success rate: {success_rate:.2f}%")

Results file does not exist. Running predictions.


Token indices sequence length is longer than the specified maximum sequence length for this model (7586 > 4096). Running this sequence through the model will result in indexing errors
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


Processed 1 items, 4001 items remaining
Processed 2 items, 4000 items remaining
Processed 3 items, 3999 items remaining
Processed 4 items, 3998 items remaining
Processed 5 items, 3997 items remaining
Processed 6 items, 3996 items remaining
Processed 7 items, 3995 items remaining
Processed 8 items, 3994 items remaining
Processed 9 items, 3993 items remaining
Processed 10 items, 3992 items remaining
Processed 11 items, 3991 items remaining
Processed 12 items, 3990 items remaining
Processed 13 items, 3989 items remaining
Processed 14 items, 3988 items remaining
Processed 15 items, 3987 items remaining
Processed 16 items, 3986 items remaining
Processed 17 items, 3985 items remaining
Processed 18 items, 3984 items remaining
Processed 19 items, 3983 items remaining
Processed 20 items, 3982 items remaining
Processed 21 items, 3981 items remaining
Processed 22 items, 3980 items remaining
Processed 23 items, 3979 items remaining
Processed 24 items, 3978 items remaining
Processed 25 items, 3977 

,text,is_true,prediction
0,HollywoodLife is turning Robert Pattinson ’s n...,0,0.0
2,The budding romance between Josh Duhamel and E...,1,1.0
3,"She may not be at the Super Bowl, but she cert...",1,0.0
4,"Do not be tardy for this dance, Friday night ...",1,1.0
5,There are a handful of rumors that the Kardash...,0,1.0
...,...,...,...
3996,A diva is the female version of a hustla…o...,0,0.0
3997,Ever wonder what Hollywood stars are whisperin...,1,0.0
3998,Julia Louis-Dreyfus just wrapped her second ro...,1,1.0
3999,She’s had enough! Cheryl Cole shut down a repo...,1,1.0


              precision    recall  f1-score   support

           0       0.40      0.46      0.43       767
           1       0.83      0.79      0.81      2502

    accuracy                           0.71      3269
   macro avg       0.62      0.63      0.62      3269
weighted avg       0.73      0.71      0.72      3269

Success rate: 45.63%


In [5]:
# LLM TEST SET
df = pd.read_csv("/Applications/AI/msc_project/data/my_llm_fake_gossipcop_test.csv")
results_path='/Applications/AI/msc_project/predictions/my_llm_fake_gossipcop_test_predictions_phi3.csv'
results_df = get_results_table(results_path=results_path, df=df)
results_df = results_df.dropna()
display(results_df)

success_rate, classwise_success_rates, metrics = get_metrics_classwise(results_df)
print(metrics)
print(f"Success rate: {success_rate:.2f}%")
print(classwise_success_rates)

Results file does not exist. Running predictions.
Processed 1 items, 138 items remaining
Processed 2 items, 137 items remaining
Processed 3 items, 136 items remaining
Processed 4 items, 135 items remaining
Processed 5 items, 134 items remaining
Processed 6 items, 133 items remaining
Processed 7 items, 132 items remaining
Processed 8 items, 131 items remaining
Processed 9 items, 130 items remaining
Processed 10 items, 129 items remaining
Processed 11 items, 128 items remaining
Processed 12 items, 127 items remaining
Processed 13 items, 126 items remaining
Processed 14 items, 125 items remaining
Processed 15 items, 124 items remaining
Processed 16 items, 123 items remaining
Processed 17 items, 122 items remaining
Processed 18 items, 121 items remaining
Processed 19 items, 120 items remaining
Processed 20 items, 119 items remaining
Processed 21 items, 118 items remaining
Processed 22 items, 117 items remaining
Processed 23 items, 116 items remaining
Processed 24 items, 115 items remaining

,label,text,news_id,seed_text,is_true,prediction
1,llm_paraphrase,"Celebrities such as Kerry Washington, Robert D...",gossipcop-1675667628,"Superheroes, rock stars, and more are sending ...",0,1.0
2,llm_rewritten,Justin Bieber has embarked on a journey of for...,gossipcop-8901380197,Justin Bieber has been on a journey of forgive...,0,1.0
3,llm_open_generation,Intriguing Allegations Surround Ben Affleck an...,gossipcop-6048462622,Some reports suggest Ben Affleck and girlfrien...,0,0.0
4,llm_open_generation,Headline: Fond Memories of 1976 Grace the Life...,gossipcop-6251256750,Eric Forman is a typical high school student g...,0,1.0
7,llm_rewritten,"According to a recent report, there are indica...",gossipcop-8511922365,Lucky number 4? Angelina Jolie could be gettin...,0,0.0
...,...,...,...,...,...,...
133,llm_rewritten,The National Association of Black Journalists ...,gossipcop-2568949699,The National Association of Black Journalists ...,0,1.0
134,llm_rewritten,Jamie Foxx is currently attending the Monster ...,gossipcop-5729276364,Jamie Foxx is enjoying himself while hanging o...,0,1.0
135,llm_rewritten,If you happen to witness a gathering of people...,gossipcop-7767457642,If you see people queueing around the block by...,0,0.0
136,llm_rewritten,Will Smith and Jada Pinkett Smith's enduring m...,gossipcop-6326387769,Will Smith and Jada Pinkett Smith’s 20-plus ye...,0,1.0


              precision    recall  f1-score   support

         0.0       1.00      0.32      0.48       104
         1.0       0.00      0.00      0.00         0

    accuracy                           0.32       104
   macro avg       0.50      0.16      0.24       104
weighted avg       1.00      0.32      0.48       104

Success rate: 31.73%
{'llm_paraphrase': 32.432432432432435, 'llm_rewritten': 30.76923076923077, 'llm_open_generation': 32.142857142857146}


/Users/jacobshort/anaconda3/envs/llama/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jacobshort/anaconda3/envs/llama/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jacobshort/anaconda3/envs/llama/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r